In [89]:
from webtableparser import WebTableParser
import pandas as pd
# import numpy as np
# import re

site = WebTableParser()
site.create('https://www.fundsexplorer.com.br/ranking', 'table table-hover')
table = site.capture()
df = site.parse(table)


In [90]:
# Dataframe manipulation should be done according with the table
df.columns = ['codigo', 'setor', 'precoatualR$', 'liqdiariaNeg', 'dividR$', 'divyield%', 'dy3macum%', 'dy6macum%', 'dy12macum%', 'dy3mmedia%', 'dy6mmedia%', 'dy12mmedia%', 'dyano%',  'varpreco%', 'rentper%', 'rentacum%', 'patrliqR$', 'vpaR$', 'p/vpaN', 'dypatr%', 'varpatr%', 'rentpatrper%', 'rentpatracum%', 'vacfisica%', 'vacfinan%', 'qtdativosN']
df = df.applymap(lambda x: str(x).replace('R$', ''))
df = df.applymap(lambda x: str(x).replace('%', ''))
df['precoatualR$'] = df['precoatualR$'].apply(lambda x: str(x).replace('.', ''))
df['patrliqR$'] = df['patrliqR$'].apply(lambda x: str(x).replace('.', ''))
df['vpaR$'] = df['vpaR$'].apply(lambda x: str(x).replace('.', ''))
df = df.applymap(lambda x: str(x).replace(',', '.'))
df['setor'] = df['setor'].apply(lambda x: str(x).replace('Ã', 'i'))
# df['setor'] = df['setor'].apply(lambda x: re.sub(r'Ã ', 'i', x)) #alternative using regex (import re needed)

In [91]:
df['codigo'] = df['codigo'].astype('string')
df['setor'] = df['setor'].astype('string')
df['precoatualR$'] = pd.to_numeric(df['precoatualR$'], errors='coerce')
df['liqdiariaNeg'] = pd.to_numeric(df['liqdiariaNeg'], errors='coerce')
df['dividR$'] = pd.to_numeric(df['dividR$'], errors='coerce')
df['divyield%'] = pd.to_numeric(df['divyield%'], errors='coerce')
df['dy3macum%'] = pd.to_numeric(df['dy3macum%'], errors='coerce')
df['dy6macum%'] = pd.to_numeric(df['dy6macum%'], errors='coerce')
df['dy12macum%'] = pd.to_numeric(df['dy12macum%'], errors='coerce')
df['dy3mmedia%'] = pd.to_numeric(df['dy3mmedia%'], errors='coerce')
df['dy6mmedia%'] = pd.to_numeric(df['dy6mmedia%'], errors='coerce')
df['dy12mmedia%'] = pd.to_numeric(df['dy12mmedia%'], errors='coerce')
df['dyano%'] = pd.to_numeric(df['dyano%'], errors='coerce')
df['varpreco%'] = pd.to_numeric(df['varpreco%'], errors='coerce')
df['rentper%'] = pd.to_numeric(df['rentper%'], errors='coerce')
df['rentacum%'] = pd.to_numeric(df['rentacum%'], errors='coerce')
df['patrliqR$'] = pd.to_numeric(df['patrliqR$'], errors='coerce')
df['vpaR$'] = pd.to_numeric(df['vpaR$'], errors='coerce')
df['p/vpaN'] = pd.to_numeric(df['p/vpaN'], errors='coerce')
df['dypatr%'] = pd.to_numeric(df['dypatr%'], errors='coerce')
df['varpatr%'] = pd.to_numeric(df['varpatr%'], errors='coerce')
df['rentpatrper%'] = pd.to_numeric(df['rentpatrper%'], errors='coerce')
df['rentpatracum%'] = pd.to_numeric(df['rentpatracum%'], errors='coerce')
df['vacfisica%'] = pd.to_numeric(df['vacfisica%'], errors='coerce')
df['vacfinan%'] = pd.to_numeric(df['vacfinan%'], errors='coerce')
df['qtdativosN'] = pd.to_numeric(df['qtdativosN'], errors='coerce')
df = df.fillna(0)  # all NaNs filled with zero
# df['liqdiariaNeg'] = df['liqdiariaNeg'].fillna(0)  # column by column if needed
# df['liqdiariaNeg'] = df['liqdiariaNeg'].replace(np.nan, 0, regex=True)  # column by column with regex
df['liqdiariaNeg'] = df['liqdiariaNeg'].astype('int64')
df['qtdativosN'] = df['qtdativosN'].astype('int64')

In [92]:
# Filters in real state funds (rsf)
rsf = df.loc[df['dy12macum%'] >= 4.00]  # 1st filter DY > 4%

In [93]:
rsf = rsf.loc[rsf['patrliqR$'] >= 500000000.00]  # 2nd filter assets > BRL 500 MM

In [94]:
rsf = rsf.loc[rsf['liqdiariaNeg'] >= 1000]  # 3rd filter tradings >= 1000/day

In [95]:
rsf = rsf.loc[rsf['p/vpaN'] <= 1.25]  # 4th filter P/VPA <= 1.25

In [96]:
# Spliting in brick and paper funds
rsf_brick = rsf.loc[rsf['qtdativosN'] >= 10]  # 5th filter >= 10 assets
rsf_paper = rsf.loc[rsf['qtdativosN'] == 0]  # 5 th filter = 0 assets

In [97]:
rsf_brick = rsf_brick.loc[rsf_brick['vacfisica%'] <= 15]  # 6th filter vacancy <= 15%

In [98]:
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
#pd.options.plotting.backend="plotly"
#py.init_notebook_mode(connected=True)
x1 = [rsf_brick['setor'],rsf_brick['codigo']]
trace01 = go.Bar(x=x1, y=rsf_brick['dy12macum%'], name='DY% 12m Acum')
trace11 = go.Bar(x=x1, y=rsf_brick['p/vpaN'], name='P/VPA')
trace21 = go.Bar(x=x1, y=rsf_brick['vacfisica%'], name='%Vacância Física', marker_color='rgb(57, 147, 123)')
data1 = [trace01, trace11, trace21]
fig1 = go.Figure(data1)
fig1.update_layout(title='ANÁLISE FIIs TIJOLOS | DY Ano >= 4%, Patr. > 500M, Neg/dia > 1000, P/VPA =< 1.25, Ativos >= 10, Vacância Física < 15%')
fig1.show()
py.plot(fig1)

'temp-plot.html'

In [99]:
#rsf_paper = rsf_paper.loc[rsf['setor'] = 'Ti­tulos e Val. Mob.']  # 7 th filter sector = credit rights

     codigo                 setor  precoatualR$  liqdiariaNeg  dividR$  \
44   KNCR11  Ti­tulos e Val. Mob.         89.50         49818     0.27   
50   VRTA11  Ti­tulos e Val. Mob.        103.95         19111     0.51   
86   KNHY11  Ti­tulos e Val. Mob.        108.69          9270     0.65   
87   KNIP11  Ti­tulos e Val. Mob.        110.80         19334     0.75   
99   MXRF11              Hi­brido         10.82        550013     0.07   
102  HFOF11  Ti­tulos e Val. Mob.        104.63         44735     0.50   
113  HGCR11  Ti­tulos e Val. Mob.        103.25         21603     0.48   
146  MGFF11  Ti­tulos e Val. Mob.         92.62         25590     0.50   
148  BCFF11  Ti­tulos e Val. Mob.         90.95         47651     0.45   
152  RECR11  Ti­tulos e Val. Mob.        101.80         16516     0.90   
157  RBRR11  Ti­tulos e Val. Mob.        100.89         10603     0.50   
158  RBRF11  Ti­tulos e Val. Mob.         96.75         30409     0.60   
160  IRDM11  Ti­tulos e Val. Mob.     

In [101]:
x = [rsf_paper['setor'],rsf_paper['codigo']]
trace0 = go.Bar(x=x, y=rsf_paper['dy12macum%'], name='DY% 12m Acum')
trace1 = go.Bar(x=x, y=rsf_paper['p/vpaN'], name='P/VPA')
trace2 = go.Bar(x=x, y=rsf_paper['varpatr%'], name='%Variação Patr.', marker_color='rgb(57, 147, 123)')
data = [trace0, trace1, trace2]
fig = go.Figure(data)
fig.update_layout(title='ANÁLISE FIIs PAPEL | DY Ano >= 4%, Patr. > 500M, Neg/dia > 1000, P/VPA =< 1.25, Ativos >= 10, Vacância Física < 15%')
fig.show()
py.plot(fig)

'temp-plot.html'